In [1]:
import pandas as pd

# Carga el archivo CSV con '|' como delimitador
df = pd.read_csv('r.csv', sep='|')

# Muestra las primeras filas del DataFrame
df.head()


,Id,Categoria,Nombre,Valoracion,Dificultad,Num_comensales,Tiempo,Tipo,Link_receta,Num_comentarios,Num_reviews,Fecha_modificacion,Ingredientes
0,74138,Recetas de Aperitivos y tapas,Receta de Baleadas hondureñas,NaN,baja,2.0,10m,NaN,https://www.recetasgratis.net/receta-de-balead...,0,0,2020-04-06,"4 trotillas de harina,2 tazas de frijoles negr..."
1,74133,Recetas de Aperitivos y tapas,Receta de Tacos de carne molida,NaN,baja,4.0,30m,Plato principal,https://www.recetasgratis.net/receta-de-tacos-...,0,0,2020-03-27,"500 gramos de carne molida,1 papa mediana,2 cu..."
2,74154,Recetas de Aperitivos y tapas,Receta de Burritos norteños,NaN,muy baja,5.0,30m,Entrante,https://www.recetasgratis.net/receta-de-burrit...,0,0,2020-03-25,"10 tortillas de harina para burritos,500 gramo..."
3,74146,Recetas de Aperitivos y tapas,Receta de Croquetas de pollo caseras de la abuela,NaN,baja,4.0,45m,Entrante,https://www.recetasgratis.net/receta-de-croque...,0,0,2020-03-24,"1 pechuga de pollo cocida y troceada,1 pellizc..."
4,74081,Recetas de Aperitivos y tapas,Receta de Masa para empanadas fritas,NaN,baja,3.0,30m,Entrante,https://www.recetasgratis.net/receta-de-masa-p...,0,0,2020-03-03,"3 tazas de harina de trigo (420 gramos),1 pizc..."


In [2]:
df = df.drop("Valoracion", axis=1)
df = df.drop("Num_comentarios", axis=1)
df = df.drop("Num_reviews", axis=1)
df = df.drop("Fecha_modificacion", axis=1)
df = df.drop("Tipo", axis=1)

In [3]:
df.head()

,Id,Categoria,Nombre,Dificultad,Num_comensales,Tiempo,Link_receta,Ingredientes
0,74138,Recetas de Aperitivos y tapas,Receta de Baleadas hondureñas,baja,2.0,10m,https://www.recetasgratis.net/receta-de-balead...,"4 trotillas de harina,2 tazas de frijoles negr..."
1,74133,Recetas de Aperitivos y tapas,Receta de Tacos de carne molida,baja,4.0,30m,https://www.recetasgratis.net/receta-de-tacos-...,"500 gramos de carne molida,1 papa mediana,2 cu..."
2,74154,Recetas de Aperitivos y tapas,Receta de Burritos norteños,muy baja,5.0,30m,https://www.recetasgratis.net/receta-de-burrit...,"10 tortillas de harina para burritos,500 gramo..."
3,74146,Recetas de Aperitivos y tapas,Receta de Croquetas de pollo caseras de la abuela,baja,4.0,45m,https://www.recetasgratis.net/receta-de-croque...,"1 pechuga de pollo cocida y troceada,1 pellizc..."
4,74081,Recetas de Aperitivos y tapas,Receta de Masa para empanadas fritas,baja,3.0,30m,https://www.recetasgratis.net/receta-de-masa-p...,"3 tazas de harina de trigo (420 gramos),1 pizc..."


In [4]:
df = df.dropna(subset=["Ingredientes"])

In [5]:
df.isnull().sum()

Id                   0
Categoria            0
Nombre               0
Dificultad        7386
Num_comensales    9305
Tiempo            9307
Link_receta          0
Ingredientes         0
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22150 entries, 0 to 22373
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              22150 non-null  int64  
 1   Categoria       22150 non-null  object 
 2   Nombre          22150 non-null  object 
 3   Dificultad      14764 non-null  object 
 4   Num_comensales  12845 non-null  float64
 5   Tiempo          12843 non-null  object 
 6   Link_receta     22150 non-null  object 
 7   Ingredientes    22150 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 1.5+ MB


In [7]:
df.shape

(22150, 8)

In [8]:
import pandas as pd
from antlr4 import *
from recipesLexer import recipesLexer
from recipesParser import recipesParser

In [9]:
# Definir una función para usar el parser de ANTLR4
def parse_ingredients(query):
    input_stream = InputStream(query)
    lexer = recipesLexer(input_stream)
    stream = CommonTokenStream(lexer)
    parser = recipesParser(stream)
    tree = parser.query()
    
    ingredients = []
    for child in tree.getChildren():
        if isinstance(child, recipesParser.IngredientContext):
            ingredients.append(child.getText().lower())
    
    return ingredients

In [10]:
# Función para filtrar recetas basadas en los ingredientes ingresados por el usuario
def filtrar_recetas(query, df):
    ingredientes = parse_ingredients(query)
    recetas_filtradas = df[df['Ingredientes'].apply(lambda x: all(ing in x.lower() for ing in ingredientes))]
    return recetas_filtradas

# Ejemplo de consulta ingresada por el usuario
consulta_usuario = "tomate, queso, albahaca, pollo"

In [11]:
# Filtrar recetas
if 'Ingredientes' in df.columns:
    recetas = filtrar_recetas(consulta_usuario, df)
    # Ajustar 'title' según el nombre de la columna en tu DataFrame, por ejemplo 'recipe_name'
    print('encontro')
else:
    print("La columna 'all_ingredients' no existe en el DataFrame. Asegúrate de haberla creado correctamente.")

encontro


In [12]:
df_recetas = pd.DataFrame(recetas, columns=['Nombre'])
df_recetas

,Nombre
3076,Receta de Pollo empanado a la napolitana
3149,Receta de Pechugas de pollo rellenas al horno
3411,Receta de Pechuga de pollo con pesto de tomate
3533,Receta de Pechugas de pollo napolitanas
7799,Receta de Ensalada templada de pollo y bacon
8472,Receta de Curanto en olla
12989,Receta de Lasaña mexicana con tortillas
13057,Receta de Lasaña cremosa de pollo
13181,Receta de Parppadelle con verduras y pollo
13279,Receta de Torta de macarrones con pollo


In [13]:
df_rec = pd.DataFrame(recetas, columns=['Ingredientes'])
df_rec

,Ingredientes
3076,"Muslos o contramuslos de pollo,4 tomates,4 cuc..."
3149,"4 Pechugas de pollo,1 bote de tomate frito,2 H..."
3411,"1 pechuga de pollo mediana.,4 unidades de toma..."
3533,"350 gramos de pechuga de pollo,4 unidades de t..."
7799,"4 unidades de pechuga de pollo,200 gramos de B..."
8472,"1½ unidades de Repollo verde,25 unidades de pa..."
12989,"16 piezas de Tortilla de maíz,1 pechuga de pol..."
13057,"2 tazas de Pechugas de pollo deshebradas,225 g..."
13181,"1 unidad de calabacín mediano,1 unidad de zana..."
13279,"250 gramos de macarrones,1 paquete de pollo (e..."


In [50]:
def buscar_recetas():
    query = ", ".join(selected_ingredients)
    if 'Ingredientes' in df.columns:
        recetas = filtrar_recetas(query, df)
        # Limpiar el Treeview
        for row in tree.get_children():
            tree.delete(row)
        if not recetas.empty:
            categorias = recetas['Categoria'].unique()
            for categoria in categorias:
                parent = tree.insert("", tk.END, values=(categoria, "", ""), open=True)
                recetas_categoria = recetas[recetas['Categoria'] == categoria]
                for index, row in recetas_categoria.iterrows():
                    tree.insert(parent, tk.END, values=("", row['Nombre'], row['Link_receta']))
        else:
            messagebox.showinfo("Sin resultados", "No se encontraron recetas con los ingredientes seleccionados.")
    else:
        messagebox.showerror("Error", "La columna 'Ingredientes' no existe en el DataFrame.")


In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter import scrolledtext

# Función para agregar ingredientes y desactivar botones
def add_ingredient(ingredient):
    if ingredient not in selected_ingredients:
        selected_ingredients.append(ingredient)
        update_ingredients_label()

def update_ingredients_label():
    ingredients_label.config(text="Ingredientes seleccionados: " + ", ".join(selected_ingredients))

# Crear ventana principal
window = tk.Tk()
window.title("Aplicación de Recetas")

# Lista para almacenar los ingredientes seleccionados
selected_ingredients = []

# Crear y posicionar los widgets
button_pollo = tk.Button(window, text="Pollo", command=lambda: add_ingredient("Pollo"))
button_zanahoria = tk.Button(window, text="Zanahoria", command=lambda: add_ingredient("Zanahoria"))
button_papa = tk.Button(window, text="Papa", command=lambda: add_ingredient("Papa"))

ingredients_label = tk.Label(window, text="Ingredientes seleccionados: ")

button_buscar = tk.Button(window, text="Buscar", command=buscar_recetas)

result_label = tk.Label(window, text="Resultados:")

# Crear Treeview para mostrar los resultados
tree = ttk.Treeview(window, columns=("Categoria", "Nombre", "Link_receta"), show='headings')
tree.heading("Categoria", text="Categoria")
tree.heading("Nombre", text="Nombre")
tree.heading("Link_receta", text="Link_receta")
tree.column("Categoria", width=100)
tree.column("Nombre", width=200)
tree.column("Link_receta", width=200)

# Posicionar widgets en la ventana usando grid
button_pollo.grid(row=0, column=0, padx=5, pady=5, sticky="ew")
button_zanahoria.grid(row=0, column=1, padx=5, pady=5, sticky="ew")
button_papa.grid(row=0, column=2, padx=5, pady=5, sticky="ew")
ingredients_label.grid(row=1, column=0, columnspan=3, padx=5, pady=5, sticky="ew")
button_buscar.grid(row=2, column=0, columnspan=3, padx=5, pady=5, sticky="ew")
result_label.grid(row=3, column=0, columnspan=3, padx=5, pady=5, sticky="ew")
tree.grid(row=4, column=0, columnspan=3, padx=5, pady=5, sticky="nsew")

# Configurar que las columnas y filas se expandan
window.grid_columnconfigure(0, weight=1)
window.grid_columnconfigure(1, weight=1)
window.grid_columnconfigure(2, weight=1)
window.grid_rowconfigure(4, weight=1)

# Iniciar el bucle principal de la ventana
window.mainloop()

In [16]:
ingredients_label

<tkinter.Label object .!label>